In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

import seaborn as sns
import nltk
nltk.download('words')
pd.set_option('max_colwidth', 40)
import os
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
sws = set(stopwords.words('english'))
punctuation = set(string.punctuation)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
                            

[nltk_data] Downloading package words to /home/ubuntu/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [96]:
df = pd.read_json('data/data.json')

In [97]:
df_text = df[['name','description']]

In [99]:
def create_target():
    df['fraud'] = df['acct_type'].map({'fraudster_event': 1,
                                   'premium': 0,
                                   'spammer_warn': 0,
                                   'fraudster': 1,
                                   'spammer_limited': 0,
                                   'spammer_noinvite': 0,
                                   'locked': 1,
                                   'tos_lock': 0,
                                   'tos_warn': 0,
                                   'fraudster_att': 1,
                                   'spammer_web': 0,
                                   'spammer': 0})
    return df

In [100]:
df = create_target()
y = df['fraud']

In [79]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import re
from string import digits
import nltk
from html.parser import HTMLParser

'''you must have nltk packages installed to run this file. To do so run the following commands:

import ntlk 
nltk.download()

A new window should open, showing the NLTK Downloader. Click on the File menu and select Change Download Directory. 
For central installation, set this to C:\nltk_data (Windows), /usr/local/share/nltk_data (Mac)'''

class MLStripper(HTMLParser):
    '''This class parses HTML from the description column
    input: string
    output: string'''
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def clean_desc(df):

    '''cleans the data. see below for documentation. adds three new columns:
    df['description_clean'] is the new cleaned description data
    df['description_html'] are the descriptions written in html
    df['description_none'] are the descriptions with no text
    '''
    
    #creates a new column for all events using HTML invites
    df['description_html'] = df['description'].apply(lambda x: True if ('</span>' or '</div>') in x else False)

    #strips HTML documentation
    df['description_clean'] = df['description'].apply(lambda x: strip_tags(x))

    #replaces artifacts from HTML tht didn't get cleaned up through the strip_tags script
    df['description_clean'] = df['description_clean'].apply(lambda x: x.replace('\r',''))
    df['description_clean'] = df['description_clean'].apply(lambda x: x.replace('\n',''))
    df['description_clean'] = df['description_clean'].apply(lambda x: x.replace('</li>',''))
    df['description_clean'] = df['description_clean'].apply(lambda x: x.replace('<li>',' '))
    
    #remove websites
    df['description_clean'] = df['description_clean'].apply(lambda x: re.sub(r'www\.\S+\.com', '', x, flags=re.MULTILINE))

    #remove websites
    df['description_clean'] = df['description_clean'].apply(lambda x: re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE))
    
    #remove email addresses
    df['description_clean'] = df['description_clean'].apply(lambda x: re.sub('\S*@\S*\s?','', x))

    #remove special characters
    df['description_clean'] = df['description_clean'].apply(lambda x: re.sub('\W+',' ', x )) 
    
    # remove numbers from string
    df['description_clean'] = df['description_clean'].apply(lambda x: re.sub(r'\d+', '', x))
    
    # lower case everything
    df['description_clean'] = df['description_clean'].apply(lambda x: x.lower())

    #removes non-english words
    words = set(nltk.corpus.words.words())
    df['description_clean'] = df['description_clean'].apply(lambda x: " ".join(w for w in nltk.wordpunct_tokenize(x) if w.lower() in words or not w.isalpha())) 

    df['description_none']= df['description_clean']==''

    return df

In [84]:
def description_cols():
    df1 = clean_desc(df_description['description'].apply(strip_tags))
    return df1

In [18]:
df_text['description'] = df_text['description'].apply(strip_tags)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [119]:
df_text = clean_desc(df_text)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [121]:
df_text['description_clean'] = \
            [''.join([word for word in doc if word not in punctuation]) for doc in description_df['description_clean']]

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [136]:
df_text['combined_description_name'] = (df_text['name'] + ' ' + df_text['description_clean'])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [91]:
tfidf1 = TfidfVectorizer(stop_words=sws,max_features=30000)
tfidf_description_vector = tfidf1.fit_transform(df_text['description_clean'])
description_word_df = pd.DataFrame(tfidf_description_vector.toarray(),columns = tfidf1.get_feature_names())

In [92]:
tfidf2 = TfidfVectorizer(stop_words=sws,max_features=20000)
tfidf_name_vector = tfidf2.fit_transform(df_text['name'])
name_word_df = pd.DataFrame(tfidf_name_vector.toarray(),columns = tfidf2.get_feature_names())

In [137]:
tfidf3 = TfidfVectorizer(stop_words=sws,max_features=30000)
tfidf_combined_vector = tfidf3.fit_transform(df_text['combined_description_name'])
combined_word_df = pd.DataFrame(tfidf_combined_vector.toarray(),columns = tfidf3.get_feature_names())

In [123]:
count1 = CountVectorizer(stop_words=sws,max_features=30000)
count_description_vector = count1.fit_transform(df_text['description_clean'])
description_count_word_df = pd.DataFrame(count_description_vector.toarray(),columns = count1.get_feature_names())

In [124]:
count2 = CountVectorizer(stop_words=sws,max_features=30000)
count_name_vector = count2.fit_transform(df_text['description_clean'])
name_count_word_df = pd.DataFrame(count_name_vector.toarray(),columns = count2.get_feature_names())

In [139]:
count3 = CountVectorizer(stop_words=sws,max_features=30000)
combined_name_vector = count3.fit_transform(df_text['combined_description_name'])
combined_count_word_df = pd.DataFrame(combined_name_vector.toarray(),columns = count3.get_feature_names())

In [106]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=400)
# description_topics = svd.fit_transform(description_word_df)
# topic_cols = ['topic_{}'.format(i) for i in range(description_topics.shape[1])]
# description_topics = pd.DataFrame(description_topics,columns=topic_cols)

pd.DataFrame(description_topics,columns)



(14337, 20894)

In [112]:
X = name_word_df
x_train,x_test,y_train,y_test = train_test_split(X,y)
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [114]:
y_pred = rf.predict(x_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.90      0.94      0.92      3116
           1       0.44      0.32      0.37       469

   micro avg       0.86      0.86      0.86      3585
   macro avg       0.67      0.63      0.64      3585
weighted avg       0.84      0.86      0.85      3585



In [115]:
X = description_word_df
x_train,x_test,y_train,y_test = train_test_split(X,y)
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [116]:
y_pred = rf.predict(x_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96      3535
           1       0.14      0.94      0.25        50

   micro avg       0.92      0.92      0.92      3585
   macro avg       0.57      0.93      0.60      3585
weighted avg       0.99      0.92      0.95      3585



In [125]:
X = description_count_word_df
x_train,x_test,y_train,y_test = train_test_split(X,y)
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [126]:
y_pred = rf.predict(x_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96      3535
           1       0.13      0.88      0.23        50

   micro avg       0.92      0.92      0.92      3585
   macro avg       0.57      0.90      0.60      3585
weighted avg       0.99      0.92      0.95      3585



In [127]:
X = name_count_word_df
x_train,x_test,y_train,y_test = train_test_split(X,y)
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [128]:
y_pred = rf.predict(x_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96      3516
           1       0.17      0.81      0.29        69

   micro avg       0.92      0.92      0.92      3585
   macro avg       0.58      0.87      0.62      3585
weighted avg       0.98      0.92      0.95      3585



In [135]:
df_text['combined_description_name'] = (df_text['name'] + ' ' + df_text['description_clean'])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [140]:
X = combined_count_word_df
x_train,x_test,y_train,y_test = train_test_split(X,y)
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [141]:
y_pred = rf.predict(x_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.97      0.94      0.95      3344
           1       0.38      0.55      0.45       241

   micro avg       0.91      0.91      0.91      3585
   macro avg       0.67      0.74      0.70      3585
weighted avg       0.93      0.91      0.92      3585



In [143]:
X = combined_word_df
x_train,x_test,y_train,y_test = train_test_split(X,y)
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [144]:
y_pred = rf.predict(x_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.97      0.94      0.95      3353
           1       0.37      0.52      0.43       232

   micro avg       0.91      0.91      0.91      3585
   macro avg       0.67      0.73      0.69      3585
weighted avg       0.93      0.91      0.92      3585

